### Use Birdy to access ROOS Demo WPS
https://github.com/roocs/roocs-wps-demo

In [1]:
from birdy import WPSClient
url = 'https://bovec.dkrz.de/ows/proxy/roocs'
#url = 'http://localhost:5000/wps'
roocs = WPSClient(url)

### Available processes

In [2]:
roocs?

Type:            WPSClient
String form:     <birdy.client.base.WPSClient object at 0x109895b20>
File:            ~/Documents/GitHub/birdhouse/birdy/birdy/client/base.py
Docstring:      
A demostrator for a WPS service for roocs.

Processes
---------

subset
    Run subsetting on climate data.

average
    Run averaging on climate data.

orchestrate
    Run a workflow
Class docstring:
Returns a class where every public method is a WPS process available at
the given url.

Example:
    >>> emu = WPSClient(url='<server url>')
    >>> emu.hello('stranger')
    'Hello stranger'
Init docstring: 
Args:
    url (str): Link to WPS provider. config (Config): an instance
    processes: Specify a subset of processes to bind. Defaults to all
        processes.
    converters (dict): Correspondence of {mimetype: class} to convert
        this mimetype to a python object.
    username (str): passed to :class:`owslib.wps.WebProcessingService`
    password (str): passed to :class:`owslib.wps.WebProcessi

### Run subset

In [3]:
roocs.subset?

Signature:
roocs.subset(
    data_ref='cmip5.output1.MOHC.HadGEM2-ES.rcp85.mon.atmos.Amon.r1i1p1.latest.tas',
    time='2085-01-01/2120-12-30',
    pre_checked=False,
    space='-5.,49.,10.,65',
    level=1000,
)
Docstring:
Run subsetting on climate data.

Parameters
----------
data_ref : string
    Data references
time : string
    Time Period
space : string
    Bounding Box
level : integer
    Level
pre_checked : boolean
    Use checked data only.

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output
File:      ~/Documents/GitHub/birdhouse/birdy/notebooks/examples/</Users/pingu/Documents/GitHub/birdhouse/birdy/birdy/client/base.py-0>
Type:      method


In [4]:
resp_subset = roocs.subset(
    data_ref='cmip5.output1.MOHC.HadGEM2-ES.rcp85.mon.atmos.Amon.r1i1p1.latest.tas',
    time='2085-01-01/2120-12-30',
)

In [5]:
resp_subset.get()

subsetResponse(
    output='https://bovec.dkrz.de/download/outputs/roocswps/142395f2-6481-11ea-bfdd-109836a7cf3a/output.nc'
)

**Show HTTP headers of output**

In [6]:
import requests
response = requests.head(resp_subset.get()[0])
response.headers

{'Server': 'nginx/1.17.5', 'Date': 'Thu, 12 Mar 2020 16:46:59 GMT', 'Content-Type': 'application/x-netcdf', 'Content-Length': '306623', 'Connection': 'keep-alive', 'Last-Modified': 'Thu, 12 Mar 2020 16:46:55 GMT', 'ETag': '"5e6a677f-4adbf"', 'Accept-Ranges': 'bytes'}

### Run average

In [7]:
roocs.average?

Signature:
roocs.average(
    data_ref='cmip5.output1.MOHC.HadGEM2-ES.rcp85.mon.atmos.Amon.r1i1p1.latest.tas',
    axes='time',
    pre_checked=False,
)
Docstring:
Run averaging on climate data.

Parameters
----------
data_ref : string
    Data references
axes : {'time', 'latitude', 'longitude'}string
    Please choose an axes for averaging.
pre_checked : boolean
    Use checked data only.

Returns
-------
output : ComplexData:mimetype:`text/plain`
    Output
File:      ~/Documents/GitHub/birdhouse/birdy/notebooks/examples/</Users/pingu/Documents/GitHub/birdhouse/birdy/birdy/client/base.py-1>
Type:      method


### Run orchestrate workflow

In [8]:
roocs.orchestrate?

Signature: roocs.orchestrate(workflow=None, mode='tree')
Docstring:
Run a workflow

Parameters
----------
workflow : ComplexData:mimetype:`application/json`
    Workflow
mode : {'tree', 'simple'}string
    Mode

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    Output
File:      ~/Documents/GitHub/birdhouse/birdy/notebooks/examples/</Users/pingu/Documents/GitHub/birdhouse/birdy/birdy/client/base.py-2>
Type:      method


**Simple Workflow Chain**

In [9]:
wf_simple = {
    'doc': "simple workflow",
    'inputs': {
        'data_ref': ['cmip5.output1.MOHC.HadGEM2-ES.rcp85.mon.atmos.Amon.r1i1p1.latest.tas'],
    },
    'steps': [
        {'subset': {"time": "2085-01-01/2120-12-30"}},
        {'subset': {"time": "2090-01-01/2100-12-30"}},
        {'average': {"axes": "time"}}
    ]}

In [10]:
import json
wf_simple_json = json.dumps(wf_simple)
wf_simple_json

'{"doc": "simple workflow", "inputs": {"data_ref": ["cmip5.output1.MOHC.HadGEM2-ES.rcp85.mon.atmos.Amon.r1i1p1.latest.tas"]}, "steps": [{"subset": {"time": "2085-01-01/2120-12-30"}}, {"subset": {"time": "2090-01-01/2100-12-30"}}, {"average": {"axes": "time"}}]}'

In [11]:
resp_wf = roocs.orchestrate(workflow=wf_simple_json, mode='simple')

In [12]:
resp_wf.get()

orchestrateResponse(
    output='https://bovec.dkrz.de/download/outputs/roocswps/206ccfe0-6481-11ea-be91-109836a7cf3a/output.nc'
)

**Workflow with Function Tree**

In [13]:
wf_tree = {
    'doc': "tree workflow",
    'inputs': {
        'tas': ['cmip5.output1.MOHC.HadGEM2-ES.rcp85.mon.atmos.Amon.r1i1p1.latest.tas'],
    },
    "outputs": {
          "output": "average_tas/output"
      },
    'steps': {
        "subset_tas_1": {
              "run": "subset",
              "in": {
                  "data_ref": "inputs/tas",
                  "time": "2085-01-01/2120-12-30"
              }
          },
          "subset_tas_2": {
              "run": "subset",
              "in": {
                  "data_ref": "subset_tas_1/output",
                  "time": "2090-01-01/2100-12-30"
              }
          },
          "average_tas": {
              "run": "average",
              "in": {
                  "data_ref": "subset_tas_2/output",
                  "axes": "time"
              }
          }
    }}

In [14]:
wf_tree_json = json.dumps(wf_tree)
wf_tree_json

'{"doc": "tree workflow", "inputs": {"tas": ["cmip5.output1.MOHC.HadGEM2-ES.rcp85.mon.atmos.Amon.r1i1p1.latest.tas"]}, "outputs": {"output": "average_tas/output"}, "steps": {"subset_tas_1": {"run": "subset", "in": {"data_ref": "inputs/tas", "time": "2085-01-01/2120-12-30"}}, "subset_tas_2": {"run": "subset", "in": {"data_ref": "subset_tas_1/output", "time": "2090-01-01/2100-12-30"}}, "average_tas": {"run": "average", "in": {"data_ref": "subset_tas_2/output", "axes": "time"}}}}'

In [15]:
resp_wf = roocs.orchestrate(workflow=wf_tree_json, mode='tree')

In [16]:
resp_wf.get()

orchestrateResponse(
    output='https://bovec.dkrz.de/download/outputs/roocswps/ae84c9c2-6481-11ea-be91-109836a7cf3a/output.nc'
)